In [50]:
import torch
import numpy as np
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

In [51]:
def load_data(filepath):
    data = pd.read_csv(filepath)
    return data

In [52]:
def prepare_data(data, feature_columns, target_column):
    features = data[feature_columns]
    target = data[target_column]

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    X_train, X_val, y_train, y_val = train_test_split(features_scaled, target, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

In [53]:
def build_model(input_features, nhead=4):  
    class TransformerModel(pl.LightningModule):
        def __init__(self):
            super().__init__()
            self.transformer_layer = nn.TransformerEncoderLayer(d_model=input_features, nhead=4, dim_feedforward=128)
            self.encoder = nn.TransformerEncoder(self.transformer_layer, num_layers=1)
            self.regressor = nn.Linear(input_features, 1)
        
        def forward(self, x):
            x = self.encoder(x)
            x = self.regressor(x)
            return x
        
        def training_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self(x)
            loss = F.mse_loss(y_hat, y)
            return loss
        
        def validation_step(self, batch, batch_idx):
            x, y = batch
            y_hat = self(x)
            loss = F.mse_loss(y_hat, y)
            self.log('val_loss', loss)
        
        def configure_optimizers(self):
            optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
            return optimizer

    return TransformerModel()


In [54]:
def train_model(model, X_train, y_train, X_val, y_val, batch_size=64, max_epochs=50):
    train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float), torch.tensor(y_train.values, dtype=torch.float))
    val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float), torch.tensor(y_val.values, dtype=torch.float))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    trainer = pl.Trainer(max_epochs=max_epochs)
    trainer.fit(model, train_loader, val_loader)

    return model

In [55]:
def plot_predictions(y_true, y_pred, n = -1, title="Prediction vs Actual Data", markersize=3):

    if n != -1:
        n = min(n, len(y_true), len(y_pred))

        y_true = y_true[-n:]
        y_pred = y_pred[-n:]

    plt.figure(figsize=(18, 3))
    plt.plot(y_true, label='Actual Values'   , marker='o', linestyle='-' , markersize=markersize)
    plt.plot(y_pred, label='Predicted Values', marker='x', linestyle='--', markersize=markersize)

    plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.7)

    plt.title(title)
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [59]:
filepath = '../input/copper_returns_5d_final.csv'
feature_columns = ['LMCADS03', 'DXY', 'SPX', 'BCOM', 'MXWD', 'XAU', 'XAG','LMCADY']  # Ajustar según necesidad
target_column = 'LMCADY_acu_5d'

In [60]:
data = load_data(filepath)
X_train, X_val, y_train, y_val = prepare_data(data, feature_columns, target_column)
X_train


array([[ 0.67278563, -0.53115652, -0.49950823, ...,  0.38457628,
         0.29738832,  0.70733539],
       [-0.70391371, -0.99542195,  1.18133841, ...,  0.5955472 ,
         0.19294894, -0.72102821],
       [-0.77451933, -0.7001329 , -0.04033294, ...,  0.45667364,
         0.16570186, -0.74776794],
       ...,
       [-1.39129872,  0.72681914,  1.25583257, ..., -0.96208851,
        -0.6041359 , -1.47491238],
       [ 0.02913854, -0.54870262,  0.64189792, ...,  0.87614555,
         0.20368924,  0.04514321],
       [ 0.10049441, -0.4143716 , -0.21433943, ..., -0.36488917,
        -0.13786646,  0.13737986]])

In [61]:
model = build_model(X_train.shape[1])
train_model(model, X_train, y_train, X_val, y_val)

c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type  

Sanity Checking:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\gapuj\miniconda3\envs\cafe\lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
C:\Users\gapuj\AppData\Local\Temp\ipykernel_26132\1143287657.py:23: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


...

C:\Users\gapuj\AppData\Local\Temp\ipykernel_26132\1143287657.py:23: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(y_hat, y)


Epoch 49: 100%|██████████| 70/70 [00:00<00:00, 161.05it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 70/70 [00:00<00:00, 104.98it/s, v_num=1]


TransformerModel(
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
    )
    (linear1): Linear(in_features=8, out_features=128, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=128, out_features=8, bias=True)
    (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
        )
        (linear1): Linear(in_features=8, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_featur